## NOTES

#### RESOURCES

In [ ]:
https://brennan.io/2016/03/02/logging-in-with-requests/
https://kazuar.github.io/scraping-tutorial/

In [1]:
import json
import pandas as pd
import csv
import datetime
import collections
import re

from pprint import pprint

from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html
import requests
from string import capwords
from titlecase import titlecase

pprint('all done')

'all done'


In [ ]:
# TODO's

# 1. User type?  E.g. admin, moderator, etc.

#### COOKIES & LOGIN

In [ ]:
NOT NECCESSARY B/C WE ARE IMPORTING PREVIOUSLY SCRAPED RAW HTML
However, see Prison Talk Scraper Login Notebook for process.

In [90]:
has_friends = 'user_profile_has_friends'
no_friends_tab = 'user_profile_no_friends_tab'
friends_tab_no_friends = 'user_profile_friends_tab_no_friends' #


html_page = open('examples/' + friends_tab_no_friends + '.html', 'r').read()
profile = BeautifulSoup(html_page, 'lxml')

#### CONTACT INFO TAB

In [91]:
#there are other places to get this but it's nice to use each column

def get_member_id(el):
    
    mid_link = el[0].select('#additional_contact_details a')[0]['href']

    mid = mid_link.split('=')[-1]

    return mid

member_id_el = profile.select('#contactinfo')
get_member_id(member_id_el) # '321024'

'451297'

#### ABOUT ME TAB

In [93]:
def get_about_me(el):
    
    questions = about_me[0].find_all('dt')
    answers = about_me[0].find_all('dd')

    fragments = ['what', 'how', 'which', 'length', 'remaining', \
              'biography', 'location', 'interests', 'occupation']

    q_and_a = collections.OrderedDict()

    for question, answer in zip(questions, answers):

        for fragment in fragments:

            if fragment in question.text.lower():

                q_and_a[fragment] = answer.text.lower()
                
    return q_and_a
        
    
about_me = profile.select('#aboutme')
get_about_me(about_me)

In [92]:
# for question, answer in get_about_me(about_me).iteritems():
#     print question + ':', answer

# what: significant other is a former inmate.
# how: google.
# which: dee is a former cdcr inmate.
# length: about 5 years
# remaining: time's up!
# biography: our love ain't nothin' but a monster with two heads....
# location: metro los angeles, ca, usa
# interests: attending sporting events
# occupation: small business owner

#### STATISTICS TAB - LAST POST

In [104]:
# POSSIBLE TO GET FROM ALL POSTS BUT IT'LL SAVE US ONE DATA MERGE

today = 'last_post_today'
yesterday = 'last_post_yesterday'
date_mdy = 'last_post_date'

last_post_page = open('examples/' + date_mdy + '.html', 'r').read()
last_post = BeautifulSoup(last_post_page, 'lxml')

def get_last_post(el):

    last_post = stats_el[0].find_all('span', {'class':'shade'}, string='Last Post:')#[0].parent

    if last_post:

        date = last_post[0].parent.a.next_sibling.strip()

        if date == 'Today':

            today = datetime.datetime.now()
            date = today.strftime("%m-%d-%Y")
            
        elif date =='Yesterday':
        
            yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
            date = yesterday.strftime("%m-%d-%Y")
    
    else:

        date = ''
        
    return date

stats_el = last_post.select('#stats')
get_last_post(stats_el)

u'04-30-2012'

#### STATISTICS TAB - GRATITUDE

In [79]:
def get_gratitude(gratitude_el):
    
    total_text = gratitude_el[0].text + ' ' + gratitude_el[1].text
    
    # u'Total Thanks: 3,122 Thanked 4,246 Times in 1,755 Posts'
    gratitude = total_text.replace('\n', '').replace('\t', '').replace('\r', '')
    
    num_thanks = gratitude.split(' ')[2] # u'3,122' 
    num_thanked = gratitude.split(' ')[4] # u'4,246'
    num_posts_thanked = gratitude.split(' ')[7] # u'1,755'

    return num_thanks, num_thanked, num_posts_thanked

stats_el = profile.select('#stats')
gratitude = stats_el[0].select('.statistics_group')[1].select('li')[:2]
get_gratitude(gratitude) #(u'3,122', u'4,246', u'1,755')

(u'3,122', u'4,246', u'1,755')

#### FRIENDS TAB

In [183]:
# # send request for friends tab with count 100
# # "member.php?tab=friends&amp;u=16480&amp;pp=100&amp;page=1#friends"
# # get number of pages, then iterate through 
# # "member.php?tab=friends&amp;u=16480&amp;pp=100&amp;page=NUM_PAGE#friends"
# # iterate through pages on usr

has_friends = 'user_profile_has_friends'
no_friends_tab = 'user_profile_no_friends_tab'
friends_tab_no_friends = 'user_profile_friends_tab_no_friends' #
multi_page_friends = 'multi_page_friends'
multi_page_friends_pp100 = 'multi_page_friends_pp100'

html_page = open('examples/' + multi_page_friends_pp100 + '.html', 'r').read()
friends_page = BeautifulSoup(html_page, 'lxml')
friends = friends_page.find_all(id='friends')

def get_friends(el):
    
    user_friends = []
    
    if(el):
        
        print 'FRIENDS TAB EXISTS!'
        
        empty_tab = el[0].find_all(string=re.compile('empty'))
        
        if(empty_tab):
            
            # # SOME HAVE EMPTY FRIENDSHIP TAB
            print 'HAS A TAB BUT HAS NO FRIENDS :('
            
        else:
            
            friends = el[0].find_all(id=re.compile('friend_mini'))
            
            print 'HERE ARE THE FRIENDS!'
            
            for friend in friends:

                user_id = friend.a['href'].split('=')[-1]
                user_name = friend.a.img['alt']
                
                user_friends.append({'user_id': user_id, 'user_name': user_name})
    else:
        # # NOT EVERYONE HAS A FRIENDSHIP TAB
        print 'NO TAB NO FRIENDS :('
        
        
    return user_friends
        
# [{'user_id': '347403', 'user_name': 'blah8705'}, ... {'user_id': '162585', 'user_name': 'Cactus'}]
get_friends(friends)

In [390]:
def get_birthdate(el):
    
    birthdate = el.text.replace('\n', '').strip()
    
    return birthdate

# # Assuming Bday is not today
# does = members[6].select('td')[8]
# doesnt = members[7].select('td')[8]
# part = members[8].select('td')[8]

# examples = [does, doesnt, part]
# [get_birthdate(example) for example in examples] # [u'June 7, 1972', u'', u'March 15']

[u'June 7, 1972', u'', u'March 15']

In [387]:
def get_profile_pic(el):
    
    if el.img:
        profile_pic = 'Y'
    else:
        profile_pic = 'N'
        
    return profile_pic
    
# Just want to know if they use one
# y = members[6].select('td')[7]
# n = members[7].select('td')[7]

# examples = [y, n]
# [get_profile_pic(example) for example in examples] # ['Y', 'N']

In [386]:
def get_avatar(el):
    
    if el.img:
        avatar = 'Y'
    else:
        avatar = 'N'
        
    return avatar

# Just want to know if they use one
# y = members[6].select('td')[6]
# n = members[7].select('td')[6]

# examples = [y, n]
# [get_avatar(example) for example in examples] # ['Y', 'N']

['Y', 'N']

In [384]:
def get_last_visit(el):

    last_visit = el.text.replace('\n', '').strip()
    
    if(last_visit == 'Today'):
        # everything else is in unicode so...
        last_visit = unicode(datetime.date.today().strftime("%m-%d-%Y"))
    
    return last_visit

# reg = members[5].select('td')[5]
# na = members[6].select('td')[5]
# today = members[7].select('td')[5]

# examples = [reg, na, today]
# [get_last_visit(example) for example in examples] # [u'07-14-2007', u'N/A', u'03-25-2017']

[u'07-14-2007', u'N/A', u'03-25-2017']

In [376]:
def get_num_posts(el):
    
    num_posts = join_date = el.text.replace('\n', '').strip()
    
    return num_posts

# data = members[7].select('td')
# get_num_posts(data[4]) # u'27,721'

u'27,721'

In [377]:
def get_join_date(el):
    
    # u'\n                03-18-2007\n               '
    join_date = el.text.replace('\n', '').strip()
    
#     join_date = 'Today' #for testing
    if(join_date == 'Today'):
        # everything else is in unicode so...
        join_date = unicode(datetime.date.today().strftime("%m-%d-%Y"))
    
    return join_date
    
# data = members[7].select('td')
# get_join_date(data[3]) # if not Today, u'03-18-2007' else u'03-25-2017' (today)

u'03-18-2007'

In [332]:
def get_hompage(el):
    
    if(el.a):
        homepage = el.a['href']
    else:
        return '' # will be filled later in pandas
    
    return homepage

# get_hompage(data[1]) # '' or 'http://www.myspace.com/leaann65'

'http://www.myspace.com/leaann65'

In [337]:
def get_username(el):

    #u'\n                   patchouli\n                  '
    username = el.a.text.replace('\n', '').strip() 
    
    return username
    
# data = members[7].select('td') 
# get_username(data[0]) #u'sidewalker'

u'sidewalker'